# Calculate Biomass from Non-tree Vegetation and Litter

# Imports and Set-up

In [1]:
# Standard Imports
import sys
import pandas as pd

# Google Cloud Imports
import pandas_gbq

In [2]:
# Util imports
sys.path.append("../../")  # include parent directory
from src.settings import GCP_PROJ_ID, CARBON_POOLS_OUTDIR, CARBON_STOCK_OUTDIR

from src.biomass_equations import vmd0003_eq1

In [3]:
# Variables
NTV_LITTER_CSV = CARBON_POOLS_OUTDIR / "saplings_ntv_litter.csv"
PLOT_INFO_CSV = CARBON_POOLS_OUTDIR / "plot_info.csv"

# BigQuery Variables
SRC_DATASET_ID = "biomass_inventory"
DATASET_ID = "carbon_stock"
IF_EXISTS = "replace"

## Load data

In [4]:
if PLOT_INFO_CSV.exists():
    plot_info = pd.read_csv(PLOT_INFO_CSV)
else:
    query = f"""
    SELECT
        * 
    FROM {GCP_PROJ_ID}.{SRC_DATASET_ID}.plot_info"""

    # Read the BigQuery table into a dataframe
    plot_info = pandas_gbq.read_gbq(query, project_id=GCP_PROJ_ID)
    plot_info.to_csv(PLOT_INFO_CSV, index=False)

In [5]:
plot_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671 entries, 0 to 670
Data columns (total 31 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   unique_id                  671 non-null    object 
 1   data_recorder              671 non-null    object 
 2   team_no                    671 non-null    int64  
 3   plot_code_nmbr             671 non-null    int64  
 4   plot_type                  671 non-null    object 
 5   sub_plot                   671 non-null    object 
 6   yes_no                     671 non-null    object 
 7   sub_plot_shift             633 non-null    object 
 8   GPS_waypt                  633 non-null    float64
 9   GPS_id                     633 non-null    float64
 10  GPS                        576 non-null    object 
 11  GPS_latitude               576 non-null    float64
 12  GPS_longitude              576 non-null    float64
 13  GPS_altitude               576 non-null    float64

In [6]:
if NTV_LITTER_CSV.exists():
    ntv_litter = pd.read_csv(NTV_LITTER_CSV)
else:
    query = f"""
    SELECT 
        * 
    FROM {GCP_PROJ_ID}.{SRC_DATASET_ID}.saplings_ntv_litter"""

    # Read the BigQuery table into a dataframe
    ntv_litter = pandas_gbq.read_gbq(query, project_id=GCP_PROJ_ID)
    ntv_litter.to_csv(PLOT_INFO_CSV, index=False)

In [7]:
ntv_litter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674 entries, 0 to 673
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   unique_id             674 non-null    object 
 1   count_saplings        589 non-null    float64
 2   litter_bag_weight     620 non-null    float64
 3   litter_sample_weight  620 non-null    float64
 4   ntv_bag_weight        620 non-null    float64
 5   ntv_sample_weight     620 non-null    float64
dtypes: float64(5), object(1)
memory usage: 31.7+ KB


# Calculate carbon stock for litter

In [8]:
# get weight of bag contents - convert grams to kg
ntv_litter["litter_biomass_kg"] = (
    ntv_litter.ntv_sample_weight - ntv_litter.ntv_bag_weight
) / 1000

In [9]:
litter = ntv_litter[["unique_id", "litter_biomass_kg"]].copy()

In [17]:
litter = vmd0003_eq1(litter, "litter_biomass_kg", 0.15, 0.37)

In [18]:
litter.rename(
    columns={
        "CO2e_per_ha": "litter_CO2e_per_ha",
        "kg_dry_matter": "litter_kg_dry_matter",
    },
    inplace=True,
)

In [28]:
litter.head(2)

,unique_id,litter_biomass_kg,litter_kg_dry_matter,litter_CO2e_per_ha
0,308D1,0.80,0.120,6.5120
1,308A1,0.06,0.009,0.4884


In [30]:
litter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674 entries, 0 to 673
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   unique_id             674 non-null    object 
 1   litter_biomass_kg     620 non-null    float64
 2   litter_kg_dry_matter  620 non-null    float64
 3   litter_CO2e_per_ha    620 non-null    float64
dtypes: float64(3), object(1)
memory usage: 21.2+ KB


## Export data and upload to BQ

In [20]:
if len(litter) != 0:
    litter.to_csv(CARBON_STOCK_OUTDIR / "litter_carbon_stock.csv", index=False)

In [21]:
# Upload to BQ
table_schema = [
    {"name": "unique_id", "type": "STRING"},
    {"name": "litter_biomass_kg", "type": "FLOAT64"},
    {"name": "litter_kg_dry_matter", "type": "FLOAT64"},
    {"name": "litter_CO2e_per_ha", "type": "FLOAT64"},
]
if len(litter) != 0:
    pandas_gbq.to_gbq(
        litter,
        f"{DATASET_ID}.litter_carbon_stock",
        project_id=GCP_PROJ_ID,
        if_exists=IF_EXISTS,
        table_schema=table_schema,
    )

100%|██████████| 1/1 [00:00<00:00, 16644.06it/s]


# Calculate carbon stock for non-tree vegetation

In [22]:
# get weight of bag contents - convert grams to kg
ntv_litter["ntv_biomass_kg"] = (
    ntv_litter.ntv_sample_weight - ntv_litter.litter_bag_weight
) / 1000

In [23]:
ntv = ntv_litter[["unique_id", "ntv_biomass_kg"]].copy()
ntv = vmd0003_eq1(ntv, "ntv_biomass_kg", 0.15, 0.47)

In [24]:
ntv.rename(
    columns={
        "CO2e_per_ha": "ntv_CO2e_per_ha",
        "kg_dry_matter": "ntv_kg_dry_matter",
    },
    inplace=True,
)

In [29]:
ntv.head(2)

,unique_id,ntv_biomass_kg,ntv_kg_dry_matter,ntv_CO2e_per_ha
0,308D1,0.80,0.120,8.2720
1,308A1,0.06,0.009,0.6204


In [25]:
ntv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674 entries, 0 to 673
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   unique_id          674 non-null    object 
 1   ntv_biomass_kg     620 non-null    float64
 2   ntv_kg_dry_matter  620 non-null    float64
 3   ntv_CO2e_per_ha    620 non-null    float64
dtypes: float64(3), object(1)
memory usage: 21.2+ KB


(None,
   unique_id  ntv_biomass_kg  ntv_kg_dry_matter  ntv_CO2e_per_ha
 0     308D1            0.80              0.120           8.2720
 1     308A1            0.06              0.009           0.6204)

In [26]:
ntv.to_csv(CARBON_STOCK_OUTDIR / "ntv_carbon_stock.csv", index=False)

In [27]:
# Upload to BQ
if len(ntv) != 0:
    pandas_gbq.to_gbq(
        ntv,
        f"{DATASET_ID}.ntv_carbon_stock",
        project_id=GCP_PROJ_ID,
        if_exists=IF_EXISTS,
    )

100%|██████████| 1/1 [00:00<00:00, 29330.80it/s]
